In [63]:
import requests
import json

rules = {
    "mechanicalrepricing": {
        'url': "https://aps-mech-time-on-inventory-repricing.azurewebsites.net/api/aps-repricing-mechanical-time-on-inventory",
        'data': {}
    }
}

rules2 = {
    "nonbodyrepricing": {
        'url': "https://aps-others-parts-function.azurewebsites.net/api/aps-others-parts-non-body-parts-comparables",
        'data': {
            "damageWords" : ["NIQ", "CHIP", "CHIPPED", "CHIPS", "CRACK", "CRACKED", "DELAMINATION",
            "DELAMINATED", "ACCESSOR", "ACCESSORIES", "ACCESSORY", "BAD", "BONDO", "CHECK", "CN",
            "MISSING", "PEEL", "PEELING", "RASH", "BUBBLE", "BUBBLED", "DENT", "FLOOD", "FLUSH",
            "H20", "H2O", "OUT", "LIFTOUT", "SET", "AM", "AFTERMKT", "AMKT", "AFTMKT", "AFTERMARKET",
            "PARTS ONLY", "FOR PARTS", "CHANGEOVER", "BUILDER", "LOCAL", "REPAIRED", "REPAINTED", "WORK",
            "DENTED", "BENT", "BROKE", "BROKEN", "DAMAGE", "DAMAGED", "DAMAGES", "GLUE", "ALUED", "BALAST ONLY",
            "BALEST ONLY", "BALLAST ONLY", "CASING ONLY", "CASING ONLY", "RIDE HEIGHT", "WET INSIDE",
            "SCRAPED UNK", "REFLECTION", "RTO", "CURR ON FE", "FOGGY", "TWIST", "PITTED", "1V2", "SENSORS ONLY",
            "RAIL ONLY", "FRAME RAIL", "APRON", "WL", "VWI", "BURNT", "BURNED", "BURN", "NEED", "NEEDED",
            "TOWER", "SETUP", "REPLICA", "DEPOT", "NAM", "REPLACEMENT", "LOOK AT", "LESS", "WELD", "FACTORY REMAN",
            "PROFESSIONAL REMAN","REMAN", "ONLY"],
            "days": "90",
            "rangeN1" : "0",
            "rangeN2" : "120",
            "rangeN3" : "202",
            "weightOneAPS" : "50",
            "weightTwoAPS" : "75",
            "weightThreeAPS" : "100",
            "defineQuartile1" : "1",
            "defineQuartile2" : "0",
            "defineQuartile3" : "4",
            "defineQuartile4" : "4"
        }
    },
    "bodyrepricing": {
        'url': "https://aps-others-parts-function.azurewebsites.net/api/aps-others-parts-body-parts-comparables?code=RzfQA00nbXW1-wEM6jHgWIRLGZZgNp-3kNqgKcglTB5DAzFulWy9Kw==",
        'data': {
            "damageWords" : ["NIQ", "CHIP", "CHIPPED", "CHIPS", "CRACK", "CRACKED", "DELAMINATION",
            "DELAMINATED", "ACCESSOR", "ACCESSORIES", "ACCESSORY", "BAD", "BONDO", "CHECK", "CN",
            "MISSING", "PEEL", "PEELING", "RASH", "BUBBLE", "BUBBLED", "DENT", "FLOOD", "FLUSH",
            "H20", "H2O", "OUT", "LIFTOUT", "SET", "AM", "AFTERMKT", "AMKT", "AFTMKT", "AFTERMARKET",
            "PARTS ONLY", "FOR PARTS", "CHANGEOVER", "BUILDER", "LOCAL", "REPAIRED", "REPAINTED", "WORK",
            "DENTED", "BENT", "BROKE", "BROKEN", "DAMAGE", "DAMAGED", "DAMAGES", "GLUE", "ALUED", "BALAST ONLY",
            "BALEST ONLY", "BALLAST ONLY", "CASING ONLY", "CASING ONLY", "RIDE HEIGHT", "WET INSIDE",
            "SCRAPED UNK", "REFLECTION", "RTO", "CURR ON FE", "FOGGY", "TWIST", "PITTED", "1V2", "SENSORS ONLY",
            "RAIL ONLY", "FRAME RAIL", "APRON", "WL", "VWI", "BURNT", "BURNED", "BURN", "NEED", "NEEDED",
            "TOWER", "SETUP", "REPLICA", "DEPOT", "NAM", "REPLACEMENT", "LOOK AT", "LESS", "WELD", "FACTORY REMAN",
            "PROFESSIONAL REMAN","REMAN", "ONLY"],
            "bodyPartsValue": ["DA", "DB", "DC", "DD", "CE", "CF", "CD", "CB", "PA", "PB", "PD", "PE", "RA", "RB"],
            "days": "90",
            "rangeN1" : "0",
            "rangeN2" : "120",
            "rangeN3" : "202",
            "weightOneAPS" : "50",
            "weightTwoAPS" : "75",
            "weightThreeAPS" : "100",
            "defineQuartile1" : "1",
            "defineQuartile2" : "0",
            "defineQuartile3" : "4",
            "defineQuartile4" : "4"
        }
    },
    "noicrepricing": {
        'url': "https://aps-noicparts-repricing-function.azurewebsites.net/api/aps-noicparts-repricing-function?code=fvOqUzp4h18AdHlePq8gg2UoTsfa1Jy7Dvs9ryofZZPfAzFu06uZPg==",
        'data': {}
    }
}

def apirequest(endpoint, data):    
    res = requests.post(url = endpoint, json = data)
    return res

def mechanicalvalidation(resmessage):
    message = resmessage.text
    inicio = message.find(":")+2
    fin = len(message)
    path = message[inicio:fin]
    res = requests.get(url=path)
    data = json.loads(res.text)
    status = data["runtimeStatus"]
    while status == "Running":
        res = requests.get(url=path)
        data = json.loads(res.text)
        status = data["runtimeStatus"]
    if(status == "Completed"):
        response = {
            "text": data["output"][0] + " " + data["output"][1],
            "status_code": 200
        }
    else:
        response = {
            "text": data["output"],
            "status_code": 500
        }
    return response
    

In [64]:
nrules = list(rules.keys())

responses = []
codes = []

for i in range(0,len(nrules)):
    res = apirequest(rules[nrules[i]]['url'],rules[nrules[i]]['data'])
    print(res)
    if (nrules[i] == "mechanicalrepricing"):
        res = mechanicalvalidation(res)
        responses.append(res["text"])
        codes.append(res["status_code"])
        print(res)
        if (res["status_code"]!=200):
            break
        continue
    responses.append(res.text)
    codes.append(res.status_code)
    if (res.status_code!=200):
        break
print("Final response")
print(responses)
print(codes)
    

<Response [200]>
{'text': 'Completed aps-mechanical-comparables-activity process Completed executing aps-mechanical-time-on-inventory-activity!', 'status_code': 200}
Final response
['Completed aps-mechanical-comparables-activity process Completed executing aps-mechanical-time-on-inventory-activity!']
[200]


In [31]:
import csv
import pandas as pd


In [56]:
a = pd.read_csv('mechanical_unique_reprice.csv',header = None, names=["uniquecode","reprice"]) 
b = pd.read_csv('noicparts_unique_reprice.csv',header = None, names=["uniquecode","reprice"]) 
c = pd.read_csv('bodyparts_unique_reprice.csv',header = None, names=["uniquecode","reprice"]) 
d = pd.read_csv('nonbodyparts_unique_reprice.csv',header = None, names=["uniquecode","reprice"]) 

print(a.shape[0])
print(a.shape[1])


1585
2


In [46]:
y = c.shape[0]
print(y)
y = c.shape[1]
print(y)
x = a.append([b,c,d],ignore_index=True)
print(x)

3488
2
      uniquecode  reprice
0     1001497702   1300.0
1     1001498242   1100.0
2     1001462092     40.0
3     1001462102     95.0
4     1001389418    250.0
...          ...      ...
3489  1001481115     75.0
3490  1001349756    295.0
3491  1001342799    245.0
3492  1001313709     55.0
3493  1001377966    195.0

[3494 rows x 2 columns]


C:\Users\Fabian-Poveda\AppData\Local\Temp\ipykernel_31820\2782567105.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x = a.append([b,c,d],ignore_index=True)


In [65]:
# Create one file with the results of the 4 functions 

import logging
import azure.functions as func
import tempfile
import os
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient
from azure.storage.blob import ContentSettings, ContainerClient

container = "apsrepricing-files"
mechanical_reprice_filename = "mechanical_unique_reprice.csv"
noic_reprice_filename = "noicparts_unique_reprice.csv"
body_reprice_filename = "bodyparts_unique_reprice.csv"
nonbody_reprice_filename = "nonbodyparts_unique_reprice.csv"

MY_CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=standardpricingstorage;AccountKey=aNkJlZtoz/HEjBwEfM4uYAcDd39JQlkyWS+10795Du/y+K0boexXPtg19mUOjnZBrJes+ESpZQGG+AStpA3e5g==;EndpointSuffix=core.windows.net"

def uploadToBlobStorage(containerName,fileName,upload_file_path):

            blob_service_client = BlobServiceClient.from_connection_string(MY_CONNECTION_STRING)
            blob_client = blob_service_client.get_blob_client(container=containerName, blob=fileName)
            
            with open(upload_file_path, "rb") as data:
                blob_client.upload_blob(data,overwrite=True, connection_timeout=14400)
                print(f"uploading file - {fileName}")

LOCAL_BLOB_PATH = tempfile.gettempdir()

def download_blob(file_name, container):
    blob_service_client =  BlobServiceClient.from_connection_string(MY_CONNECTION_STRING)
    my_container = blob_service_client.get_container_client(container)
    print(file_name)
    bytes = my_container.get_blob_client(file_name).download_blob().readall()
    # Get full path to the file
    download_file_path = os.path.join(LOCAL_BLOB_PATH, file_name)
    # for nested blobs, create local path as well!
    os.makedirs(os.path.dirname(download_file_path), exist_ok=True)
    with open(download_file_path, "wb") as file:
        file.write(bytes)
        return download_file_path

def saveToCSV_no_header(df, fileName):
    file_path = os.path.join(LOCAL_BLOB_PATH, fileName)
    df.to_csv(file_path, index=False, header = False)
    return file_path

def read_reprice_file(file_name, container):
    file_path = download_blob(file_name, container)
    reprice_file = pd.read_csv(file_path)
    file = pd.DataFrame()
    file['unique'] = None
    file['reprice'] = None
    if reprice_file.shape[0] > 0:
        repricefile = pd.read_csv(file_path, header = None, names=["unique","reprice"])
        file = file.append(repricefile,ignore_index=True)
    return file 


In [68]:
mechanicalfile = read_reprice_file(mechanical_reprice_filename, container)
noicfile = read_reprice_file(noic_reprice_filename, container)
bodyfile = read_reprice_file(body_reprice_filename, container)
nonbodyfile = read_reprice_file(nonbody_reprice_filename, container)
mainfile = pd.DataFrame()
mainfile['unique'] = None
mainfile['reprice'] = None
repricefile = mainfile.append([mechanicalfile, bodyfile, nonbodyfile, noicfile], ignore_index=True)

mechanical_unique_reprice.csv


C:\Users\Fabian-Poveda\AppData\Local\Temp\ipykernel_31820\1010714240.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file = file.append(repricefile,ignore_index=True)


noicparts_unique_reprice.csv


C:\Users\Fabian-Poveda\AppData\Local\Temp\ipykernel_31820\1010714240.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file = file.append(repricefile,ignore_index=True)


bodyparts_unique_reprice.csv


C:\Users\Fabian-Poveda\AppData\Local\Temp\ipykernel_31820\1010714240.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file = file.append(repricefile,ignore_index=True)


nonbodyparts_unique_reprice.csv


C:\Users\Fabian-Poveda\AppData\Local\Temp\ipykernel_31820\1010714240.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  file = file.append(repricefile,ignore_index=True)


bodyparts_manualreprice.csv


C:\Users\Fabian-Poveda\AppData\Local\Temp\ipykernel_31820\777353175.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  repricefile = mainfile.append([mechanicalfile, bodyfile, nonbodyfile, noicfile, nonbodyfile2], ignore_index=True)


In [69]:
reprice_csv_path = saveToCSV_no_header(repricefile, 'aps_unique_reprice.csv')
uploadToBlobStorage(container,'aps_unique_reprice.csv',reprice_csv_path)

uploading file - aps_unique_reprice.csv
